In [ ]:
!gpustat

# Importing Packages

In [5]:
import os
import re
import sys
import math
from math import floor
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from pandas_summary import DataFrameSummary
from pandas.api.types import is_numeric_dtype, is_categorical_dtype
import IPython
from IPython.display import display, clear_output
from PIL import Image
import imgaug as ia
from imgaug import augmenters as iaa
import imageio
import matplotlib
import cv2
import shutil
import glob
import mlflow
from mlflow import log_metric, log_param, log_artifact
from  mlflow.tracking import MlflowClient
import mlflow.keras
# import fastai
# # fastai.__version__ # '1.0.58'
# from fastai.imports import *
# from fastai.structured import *

from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.feature_extraction import FeatureHasher
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
le = preprocessing.LabelEncoder()
from sklearn import metrics
from sklearn.tree import export_graphviz
from sklearn.utils import shuffle, resample
from sklearn.externals import joblib
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score, classification_report, confusion_matrix
from sklearn.metrics import roc_curve, precision_recall_curve, auc
from sklearn.utils.class_weight import compute_class_weight

import time, sys
import graphviz

from keras.models import Sequential, Model, load_model
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv1D, Conv2D, MaxPooling1D, MaxPooling2D
from keras.layers.core import Activation,Dense,Dropout
from keras.layers import Input, Flatten, GlobalAveragePooling2D, concatenate
from keras.optimizers import Adam, RMSprop, Adamax, SGD
from keras.utils import to_categorical, Sequence
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import MobileNet, VGG16, resnet50
from keras.applications.mobilenet import preprocess_input
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping, ReduceLROnPlateau
from keras import regularizers
from keras.regularizers import l2
from keras import models, layers, optimizers

# from pyimagesearch import datasets
# from pyimagesearch import models

%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Fundus Data Loading

In [2]:
ROOT_dir = "/ds/notebooks/patra/"
DATA_dir = "/ds2/data/retina/UK_BB/patra/"
tts_path = DATA_dir+'TrainTestVal/'

In [3]:
fundus_df = pd.read_feather('/ds2/data/retina/UK_BB/UK_BB_DATA/fundus.feather')
# Checking data size
n_patients = len(fundus_df)
n_features = len(fundus_df.columns)
print("Dataset consist of %d patients data with %d features for each patient" %(n_patients, n_features))

df_fundus_left = fundus_df.iloc[:,5613:5617] # fundus images of left eye
df_fundus_right = fundus_df.iloc[:,5617:5621] # fundus images of right eye

Dataset consist of 85730 patients data with 6251 features for each patient


# Function Definitions

In [4]:
# Converting the disease code values to integer
def conv2int(tempdf):
    float_col = tempdf.select_dtypes(include = ['float64']) # This will select float columns only
    for col in float_col.columns.values:
        tempdf[col] = tempdf[col].fillna(0.0).astype('int64')
    return tempdf

#Getting unique items and their frequency
def get_count(data):
    data_freq = []
    data_unique = []
    for i in range(data.shape[0]):
        uniqueValues, occurCount = np.unique(data[i], return_counts=True)
        data_freq.append([uniqueValues, occurCount]) # for each patient
        data_unique.append(uniqueValues.tolist()) # unique disease for each patient
    return data_freq, data_unique

# Obtaining unique disease occurance for each patient in a sorted array
def get_sorted_unique(df):
    df_int = conv2int(df)
    arr = df_int.to_numpy()
    data_freq, data_unique = get_count(arr)
    uniqueValues, occurCount = np.unique(np.asarray([item for sublist in data_unique for item in sublist]), return_counts=True)
    disease_freq = {uniqueValues[i]: occurCount[i] for i in range(len(occurCount))}
    sort_disease_freq = sorted(disease_freq.items(), key=lambda x: x[1], reverse=True)
    return sort_disease_freq

# Displaying all values in a dataframe
def display_all(df):
    with pd.option_context("display.max_rows",1000):
        with pd.option_context("display.max_columns",1000):
            display(df)

            
# To categorise the diseases into 4 categories as per disease_types dictionary
def categorize(sort_disease_freq):
    dis_cat = []
    for i in range(len(sort_disease_freq)):
        flag = False
        for j in range(len(k)):
    #         print(sort_disease_freq[i][0], v[j])
            if(sort_disease_freq[i][0] in v[j]):
                flag = True
                flagv = j
            else:
                continue
        if (flag == True):
            dis_cat.append(k[flagv])
        else:
            dis_cat.append("Others")
            
    return dis_cat

def which_eye(patient):
    s = df_100515.iloc[patient]
    t = s.notna()
    f = s[t==True]
    lf = len(f)
    if(lf==1):
        val = f[0]
    elif(lf>1):
#         print("which_eye: Multiple answers for patient:",patient,"-",lf)
#         print(f)
        val = f[lf-1]
    else:
        val = 0
    return val


def get_img_name(patient,df):
    s = df.iloc[patient]
    t = s.notna()
    f = s[t==True]
    lf = len(f)
    if(lf==1):
        txt = f[0]
    elif(lf>1):
#         print("get_img_name: Multiple answers for patient:",patient,"-",lf)
#         print(f)
        txt = f[lf-1]
    else:
        txt = 'None'
    prefix = fundus_df.iloc[patient]['eid']
    img_name = str(prefix) + '_' + txt + '.png'
        
    return img_name    

# convert image to array of size "img_size" and normalize by diving each pixel by 255
def img2arr(img, img_size):
    imgarr = cv2.imread(img)
    imgarr = cv2.resize(imgarr,(img_size,img_size), interpolation=cv2.INTER_CUBIC)
    imgarr = cv2.cvtColor(imgarr, cv2.COLOR_BGR2RGB)
    return imgarr


# To run: update_progress(iter/total no. of elements)
def update_progress(progress):
    bar_length = 20
    if isinstance(progress, int):
        progress = float(progress)
    if not isinstance(progress, float):
        progress = 0
    if progress < 0:
        progress = 0
    if progress >= 1:
        progress = 1

    block = int(round(bar_length * progress))

    clear_output(wait = True)
    text = "Progress: [{0}] {1:.1f}%".format( "#" * block + "-" * (bar_length - block), progress * 100)
    print(text)
    

## GROUND TRUTH CREATION:

def GT_100523():
    GT_100523 = [0] * len(fundus_df)
    # 1: Glaucoma
    # 0: No Glaucoma

    cnt = 0
    cnt1 = 0
    cnt2 = 0
    for i in range(len(fundus_df)):
        if(2 in df100523_unique_arr[i]): # 2 means Glaucoma
            GT_100523[i] = 1
            cnt+=1
        elif(-1 in df100523_unique_arr[i] or -3 in df100523_unique_arr[i]): # -1 don't know, -3 prefer not to answer
            GT_100523[i] = 99
            cnt1+=1
        else:
            cnt2+=1

    print("In DC 100523 : \n")
    print("No. of patients having glaucoma : ",cnt)
    print("Patients who don't know or prefer not to answer: ",cnt1)
    print("Healthy patients:", cnt2)
    
    return GT_100523

def GT_19():
    GT_19 = [0] * len(fundus_df)
    # 1: Glaucoma
    # 0: No Glaucoma

    cnt = 0
    for i in range(len(fundus_df)):
        if (len(df19_clean_arr[i])>1): # len=1 contains nan only 
            for j in range(len(df19_clean_arr[i])):
                if(('H40' in df19_clean_arr[i][j]) or ('H42' in df19_clean_arr[i][j])): # H40 and H42 refers to glaucoma related patients
    #                 print(i, df19_clean_arr[i])
                    GT_19[i] = 1 # 1 is assigned to those having glaucoma as per code 100523 and icd10
                    cnt+=1
                    break

    print("No. of patients having glaucoma as per DC19 / ICD10 are: ",cnt)
    return GT_19

def GT_intersect():
    GT_intersect = [0] * len(fundus_df)

    # 1: Glaucoma as per both
    # 0: No Glaucoma as per both
    # 2: glau_19_not100523
    # 3: glau_100523_not19

    for i in range(len(fundus_df)):
        if(2 in df100523_unique_arr[i]): # 2 means Glaucoma
            if (len(df19_clean_arr[i])>1): # len=1 contains nan only 
                for j in range(len(df19_clean_arr[i])):
                    if(('H40' in df19_clean_arr[i][j]) or ('H42' in df19_clean_arr[i][j])): # H40 and H42 refers to glaucoma related patients
                        # print(i, df100523_unique_arr[i],"=+=+=",df19_clean_arr[i])
                        GT_intersect[i] = 1 # 1 is assigned to those having glaucoma as per code 100523 and icd10
                        break
                    # Rows mismatching in the datasets
                    elif(j==len(df19_clean_arr[i])-1):
                        GT_intersect[i] = 3
            else:
                GT_intersect[i] = 3
        else:
            # Rows mismatching in the datasets
            if(len(df19_clean_arr[i])>1): # len=1 contains nan only 
                for j in range(len(df19_clean_arr[i])):
                    if(('H40' in df19_clean_arr[i][j]) or ('H42' in df19_clean_arr[i][j])): # H40 and H42 refers to glaucoma related patients
                        GT_intersect[i] = 2 # not in 100523, but in icd10
                        break

    glau_100523_19 = []
    no_glau_100523_19 = []
    glau_100523_not19 = []
    glau_19_not100523 = []

    for i in range(len(GT_intersect)):
        if(GT_intersect[i]==0):
            no_glau_100523_19.append(i)
        elif(GT_intersect[i]==1):
            glau_100523_19.append(i)
        elif(GT_intersect[i]==2):
            glau_19_not100523.append(i)
        elif(GT_intersect[i]==3):
            glau_100523_not19.append(i)

    print("Glaucoma in both:",len(glau_100523_19),"\nGlaucoma in none:",len(no_glau_100523_19),
          "\nGlaucoma in 100523 only:",len(glau_100523_not19),"\nGlaucoma in icd10 only:",len(glau_19_not100523))
    
    return GT_intersect

def GT_union():
    GT_union = [0] * len(fundus_df)
    cnt = 0

    for i in range(len(fundus_df)):
        if(2 in df100523_unique_arr[i]):
            GT_union[i] = 1
            cnt+=1
        else:
            if(len(df19_clean_arr[i])>1): # len=1 contains nan only 
                for j in range(len(df19_clean_arr[i])):
                    if(('H40' in df19_clean_arr[i][j]) or ('H42' in df19_clean_arr[i][j])): # H40 and H42 refers to glaucoma related patients
                        GT_union[i] = 1
                        cnt+=1
                        break
                    elif(j == (len(df19_clean_arr[i])-1)):
                        if(-1 in df100523_unique_arr[i] or -3 in df100523_unique_arr[i]): # -1 don't know, -3 prefer not to answer
                            GT_union[i] = 99
                         
    print("No. of patients having glaucoma as per union of 100523 and ICD10 are: ",cnt)
    
    return GT_union

def GT_list2df_merge(gt1,gt2,gt3,gt4):
    # Convert Ground truths to dataframe
    gt1_df = pd.DataFrame(gt1, columns=['GT_100523']) 
    gt2_df = pd.DataFrame(gt2, columns=['GT_19']) 
    gt3_df = pd.DataFrame(gt3, columns=['GT_intersect']) 
    gt4_df = pd.DataFrame(gt4, columns=['GT_union'])

    allGT_df = pd.concat([gt1_df, gt2_df, gt3_df, gt4_df], axis=1)
    
    return allGT_df

## FEATURE IMPORTANCE RELATED

def proc_df(orig_df,orig_y):
    '''
    Separates the dataframe into dependent df and independent y variables
    '''
    flag = 0
    df = orig_df.copy()
    y = df[orig_y].values
    df.drop(orig_y, axis=1, inplace=True)
    return df,y
# df, y = proc_df(fundus_gt,'Glaucoma_GT')

def rmse(x,y):
    return math.sqrt(((x-y)**2).mean())

def print_score(rf, train_x, val_x, train_y, val_y):
    res = [rf.score(train_x, train_y), 
           rmse(rf.predict(train_x),train_y), 
           rf.score(val_x, val_y), 
           rmse(rf.predict(val_x),val_y)]
    if hasattr(rf, 'oob_score_'):
        res.append(rf.oob_score_)
    print(res)
    
# copied from fastai.structured
def draw_tree(t, df, size=10, ratio=0.6, precision=0):
    """ Draws a representation of a random forest in IPython.
    Parameters:
    -----------
    t: The tree you wish to draw
    df: The data used to train the tree. This is used to get the names of the features.
    """
    s=export_graphviz(t, out_file=None, feature_names=df.columns, filled=True,
                      special_characters=True, rotate=True, precision=precision)
    IPython.display.display(graphviz.Source(re.sub('Tree {',
       f'Tree {{ size={size}; ratio={ratio}', s)))

# copied from fastai.structured
# fi = rf_feat_importance(RF,enc_df) -- NOT USED
def rf_feat_importance(m, df):
    return pd.DataFrame({'cols':df.columns, 'descrip':df.columns.to_series().map(feat_desc_dict),'importance':m.feature_importances_}
                       ).sort_values('importance', ascending=False)
def plot_fi(finew):
    return finew.plot('Features','Importance','barh',figsize=(15,15),legend=False)

## DESCRIPTION OF FEATURES - DICTIONARY
def feature_desc():
    feat_desc_dict = dict([(k,v) for k,v in zip(feat_desc.UDI,feat_desc.Description)])  
    feat_unique_val = []
    feat_unique_val.append(feat_desc_dict['eid'])
    for i in range(1,len(feat_unique)):
        for key, value in feat_desc_dict.items():   # iter on both keys and values
            if key.startswith(feat_unique[i]+'-'):
                feat_unique_val.append(value)
                break
    feat_unique_val = np.asarray(feat_unique_val)
    feat_unique_dict = dict(zip(feat_unique, feat_unique_val))
    
    return feat_desc_dict, feat_unique_dict

## IMPORTANCE OF FEATURES - DICTIONARY
def feature_imp(RF, enc_df):    
    imp = RF.feature_importances_.tolist()
    col = enc_df.columns.values.tolist()
    feat_imp_dict = dict(zip(col, imp))

    feat_unique_impval = []
    if(feat_unique[0] in feat_imp_dict.keys()):
        feat_unique_impval.append(feat_imp_dict[feat_unique[0]])
    else:
        feat_unique_impval.append(0)

    for i in range(1,len(feat_unique)):
        temp = []
        for key, value in feat_imp_dict.items():   # iter on both keys and values
            if key.startswith(feat_unique[i]+'-'):
                temp.append(feat_imp_dict[key])
        if(len(temp) > 0):
            feat_unique_impval.append(max(temp))
        else:
            feat_unique_impval.append(0)

    feat_unique_imp_dict = dict(zip(feat_unique, feat_unique_impval))
    
    return feat_imp_dict, feat_unique_imp_dict

def get_imp_features(num): # returns the dataframe of features along with their importances
    feat = []
    imp = []
    imp_sorted = sorted(feat_unique_imp_dict.items(), key=lambda x: x[1], reverse=True)
    for i in range(num):
        feat.append(feat_unique_dict[imp_sorted[i][0]])
        imp.append(imp_sorted[i][1])
    
    return pd.DataFrame(list(zip(feat, imp)), 
               columns =['Features', 'Importance'])



# Plot the training and validation loss + accuracy
def plot_training(history):
    # summarize history for accuracy  

    plt.subplot(211)  
    plt.plot(history.history['accuracy'])  
    plt.plot(history.history['val_accuracy'])  
    plt.title('model accuracy')  
    plt.ylabel('accuracy')  
    plt.xlabel('epoch')  
    plt.legend(['train', 'test'], loc='upper left')  

    # summarize history for loss  

    plt.subplot(212)  
    plt.plot(history.history['loss'])  
    plt.plot(history.history['val_loss'])  
    plt.title('model loss')  
    plt.ylabel('loss')  
    plt.xlabel('epoch')  
    plt.legend(['train', 'test'], loc='upper left')  
    plt.show()  
    

colors = plt.rcParams['axes.prop_cycle'].by_key()['color'] 

def plot_cm(labels, predictions, p=0.5):
    cm = confusion_matrix(labels, predictions > p)
    normed_cm = (cm.T / cm.astype(np.float).sum(axis=1)).T

    plt.figure(figsize=(5,5))
    sns.heatmap(normed_cm, annot=True, fmt="f",cbar=True, cmap="Blues")
    plt.title('Confusion matrix @{:.2f}'.format(p))
    plt.ylabel('Actual label')
    plt.xlabel('Predicted label')

    print('Healthy Patients - correctly identified (True Negatives): ', cm[0][0], normed_cm[0][0])
    print('Healthy Patients - predicted to have glaucoma (False Positives): ', cm[0][1], normed_cm[0][1])
    print('Glaucoma Patients - predicted healthy (False Negatives): ', cm[1][0], normed_cm[1][0])
    print('Glaucoma Patients - correctly identified (True Positives): ', cm[1][1], normed_cm[1][1])
    print('\nTotal Glaucoma patients: ', np.sum(cm[1]), np.sum(normed_cm[1]))
    print('Total Healthy patients: ', np.sum(cm[0]), np.sum(normed_cm[0]))


def print_confusion_matrix(confusion_matrix, class_names, figsize = (10,7), fontsize=14):
    """Prints a confusion matrix, as returned by sklearn.metrics.confusion_matrix, as a heatmap.
    
    Arguments
    ---------
    confusion_matrix: numpy.ndarray
        The numpy.ndarray object returned from a call to sklearn.metrics.confusion_matrix. 
        Similarly constructed ndarrays can also be used.
    class_names: list
        An ordered list of class names, in the order they index the given confusion matrix.
    figsize: tuple
        A 2-long tuple, the first value determining the horizontal size of the ouputted figure,
        the second determining the vertical size. Defaults to (10,7).
    fontsize: int
        Font size for axes labels. Defaults to 14.
        
    Returns
    -------
    matplotlib.figure.Figure
        The resulting confusion matrix figure
    """
    df_cm = pd.DataFrame(
        confusion_matrix, index=class_names, columns=class_names, 
    )
    fig = plt.figure(figsize=figsize)
    heatmap = sns.heatmap(df_cm, annot=True, fmt="f", cmap="Blues")
    heatmap.yaxis.set_ticklabels(heatmap.yaxis.get_ticklabels(), rotation=0, ha='right', fontsize=fontsize)
    heatmap.xaxis.set_ticklabels(heatmap.xaxis.get_ticklabels(), rotation=45, ha='right', fontsize=fontsize)
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    
def plot_metrics(history):
    metrics =  ['loss','Accuracy', 'Precision', 'Recall', 'F1', 'AUC']
    
    for n, metric in enumerate(metrics):
        name = metric.replace("_"," ").capitalize()
        
        plt.subplot(3,2,n+1)
        plt.plot(history.epoch,  history.history[metric], color=colors[0], label='Train')
        plt.plot(history.epoch, history.history['val_'+metric],
                 color=colors[0], linestyle="--", label='Validation')
        plt.xlabel('Epoch')
        plt.ylabel(name)
#         if metric == 'loss':
#             plt.ylim([0, plt.ylim()[1]])
#         else:
        plt.ylim([0,1])
        plt.legend()
        
def plot_roc(name, labels, predictions, **kwargs):
    fp, tp, _ = roc_curve(labels, predictions)

    plt.plot(100*fp, 100*tp, label=name, linewidth=2, **kwargs)
    plt.xlabel('False positives [%]')
    plt.ylabel('True positives [%]')
    plt.grid(True)
    ax = plt.gca()
    ax.set_aspect('equal')
    
def plot_loss(history, label, n):
    # Use a log scale to show the wide range of values.
    plt.semilogy(history.epoch,  history.history['loss'],
               color=colors[n], label='Train '+label)
    plt.semilogy(history.epoch,  history.history['val_loss'],
          color=colors[n], label='Val '+label,
          linestyle="--")
    plt.xlabel('Epoch')
    plt.ylabel('Loss')

    plt.legend()
    
def get_ROI(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_OTSU + cv2.THRESH_BINARY)[1]

    # Find contour and sort by contour area
    cnts = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = cnts[0] if len(cnts) == 2 else cnts[1]
    if(len(cnts)>0):
        cnts = sorted(cnts, key=cv2.contourArea, reverse=True)

        # Find bounding box and extract ROI
        for c in cnts:
            x,y,w,h = cv2.boundingRect(c)
            ROI = image[y:y+h, x:x+w]
            break
    else:
        ROI = image
        
    return ROI

def Recall(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def Precision(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def F1(y_true, y_pred):
    precision = Precision(y_true, y_pred)
    recall = Recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

def get_metrics(y_train, y_val, y_test, train_pred, val_pred, test_pred):
    print("Metrics:\t\tTrain:\t\t\tValidation:\t\tTest:")
    print("Accuracy:\t",accuracy_score(y_train, train_pred), "\t", accuracy_score(y_val, val_pred), "\t", accuracy_score(y_test, test_pred))
    print("Recall:\t\t",recall_score(y_train, train_pred), "\t", recall_score(y_val, val_pred), "\t", recall_score(y_test, test_pred))
    print("Precision:\t",precision_score(y_train, train_pred), "\t", precision_score(y_val, val_pred), "\t", precision_score(y_test, test_pred))
    print("F1 score:\t",f1_score(y_train, train_pred), "\t", f1_score(y_val, val_pred), "\t", f1_score(y_test, test_pred))

    fpr_train, tpr_train, thresholds_train = roc_curve(y_train, train_pred)
    fpr_val, tpr_val, thresholds_val = roc_curve(y_val, val_pred)
    fpr_test, tpr_test, thresholds_test = roc_curve(y_test, test_pred)
    print("AUC:\t\t",auc(fpr_train, tpr_train), "\t", auc(fpr_val, tpr_val), "\t", auc(fpr_test, tpr_test))
    
def vgg_model():       
    vgg_conv = VGG16(weights='imagenet', include_top=False, input_shape=(red_size, red_size, 3))

    # Freeze the layers except the last 4 layers
    for layer in vgg_conv.layers[:-4]:
        layer.trainable = False
        
    # Create the model
    model = models.Sequential()
     # Add the vgg convolutional base model
    model.add(vgg_conv)
 
    # Add new layers
    model.add(layers.Flatten())
    model.add(layers.Dense(512, activation='relu',kernel_regularizer=l2(reg_value)))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(1, activation='sigmoid'))
 
    return model

def resnet50_model():
    resnet_model = resnet50.ResNet50(weights='imagenet', include_top=False, input_shape=(red_size, red_size, 3))
    # UnFreeze last conv layer
    for layer in resnet_model.layers[:158]:
        layer.trainable = False
    for layer in resnet_model.layers[158:]:
        layer.trainable = True
        
     # Create the model
    model = models.Sequential()
    model.add(resnet_model)
 
    # Add new layers
    # https://adventuresinmachinelearning.com/global-average-pooling-convolutional-neural-networks/
    model.add(layers.GlobalAveragePooling2D())
    model.add(layers.Dense(1, activation='sigmoid'))
 
    return model

def mobilenet_model():       
    mobile_conv = MobileNet(weights='imagenet', include_top=False, input_shape=(red_size, red_size, 3))

    # UnFreeze last conv layer
    for layer in mobile_conv.layers[:78]:
        layer.trainable = False
    for layer in mobile_conv.layers[78:]:
        layer.trainable = True
        
    # Create the model
    model = models.Sequential()
    model.add(mobile_conv)
 
    # Add new layers
    # https://adventuresinmachinelearning.com/global-average-pooling-convolutional-neural-networks/
    model.add(layers.GlobalAveragePooling2D())
    model.add(layers.Dense(1, activation='sigmoid'))
 
    return model

def save_mlflow(keras_model_path,exp_name,which_gt,which_model,seq,BATCH_SIZE,n_epochs,n_steps_per_epoch,n_validation_steps,reg_value,learning_rate,decay,
                opt,history,results_tr,results_vl,train_cm,val_cm):
    # Create Experiment
    exp_id = mlflow.create_experiment(exp_name)

    # Start run as a child of that experiment
    mlflow.start_run(experiment_id=exp_id)

    # Save parameters
    mlflow.log_param("which_gt", which_gt)
    mlflow.log_param("which_model", which_model)
    mlflow.log_param("augmentation", seq)
    mlflow.log_param("BATCH_SIZE", BATCH_SIZE)
    mlflow.log_param("epochs", n_epochs)
    mlflow.log_param("steps_per_epoch", n_steps_per_epoch)
    mlflow.log_param("validation_steps", n_validation_steps)
    mlflow.log_param("reg_value", reg_value)
    mlflow.log_param("learning_rate", learning_rate)
    mlflow.log_param("decay", decay)
    mlflow.log_param("optimizer", opt)
    
    # Save Model
    mlflow.keras.save_model(model, keras_model_path)

    # Save Metrics
    for i in history.history.keys():
        for j in range(len(history.history[i])):
            mlflow.log_metric(i, history.history[i][j], step=j)

    for i in ['results_tr','results_vl','train_cm','val_cm']:
        for j in range(len(eval(i))):
            mlflow.log_metric(i, eval(i)[j], step=j)
            
    mlflow.end_run()
    
def get_notebook_name():
    from IPython.core.display import Javascript
    from IPython.display import display

    display(Javascript('Jupyter.notebook.kernel.execute(\"this_notebook = " + "\'"+Jupyter.notebook.notebook_name+"\'");'))

    # os.path.abspath(this_notebook)
    return this_notebook


print("All Function definitions executed")


Function definitions executed
